Rodzaje ruchu:\
Swap - losujemy dwa miasta i zamieniamy je ze sobą.\
Insercja - losujemy miasto i punkt w którym je umieszczamy.\
Zamiana kolejności - losujemy dwa punkty i między nimi zamieniamy kolejność.\
\
Sąsiedztwo to dwa przypadki które różnią się między sobą jednym ruchem\
\
Klasyczny algorytm wspinaczki z multistartem: po każdym ruchu liczymy drogę i sprawdzamy czy jest mniejsza czy nie, wybieramy tylko lepsze opcje. Multistart polega na wielokrotnym odpaleniu algorytmu z losowaniem punktu startowego.\
\
Algorytm symulowanego wyżarzania: zmodyfikowany algorytm wspinaczki, jeśli znalezione rozwiązanie jest gorsze to nadal je rozważamy, na podstawie tego jak bardzo jest gorsze oraz od temperatury.\
Temperatura jest wysoka na początku działania algorytmu i spada razem z czasem wykonywania algorytmu, im niższa temperatura tym mniejsze prawdopodobieństwo wybrania gorszego rozwiązania, bo algorytm już trochę działa i zakładamy że jest w lepszym miejscu niż na początku.\
Pozwala on nam wyjść z niektórych ekstremów lokalnych dzięki możliwości przyjęcia gorszego rozwiązania.\
\
Tabu search:\
Sąsiedztwo wygląda tutaj inaczej: przy swapie losujemy jedną liczbę i zamieniamy wszystkie kombinacje z tą liczbą, ze wszystkich kombinacji wybieramy najlepszą.\
Następnie taka kombinacja trafia na listę tabu, czyli takiej samej zamiany nie można wykonać przez określoną kolejną liczbę ruchów.\
\
Funkcja aspiracji to funkcja, która pod pewnymi warunkami pozwala wykonać ruch z listy tabu ale nie trzeba tego implementować.\
\
Odpowiednia długość listy tabu pozwala wyjść z niektórych ekstremów lokalnych, jeśli będzie za krótka to możemy nie wyjść z ekstremum lokalnego, jeśli będzie za długa to możemy wyjść z ekstremum niepotrzebnie.
